<a href="https://colab.research.google.com/github/dwittaker/CSE535/blob/master/KERAS_Test_CSE535_Project_2_post_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Header

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 08:38:55 2020

@author: ASU CSE535-Team18. Travis Robinson, Asim Mushtaq, Miguel Turner, Duane Wittaker
"""

import math
from os import walk
from os import path
import json
import pandas as pd
import numpy as np
import datetime
import pywt
from scipy.fftpack import fft
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import sklearn.preprocessing as skprep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pickle
import keras
import keras.utils
from keras.optimizers import Adam
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM
#from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('expand_frame_repr', True)
path_testdata = '/content/drive/My Drive/poseclassifier/test'
path_trainingdata = '..\\..\\flaskr\\uploads'
path_trainingdata2 = '/content/drive/My Drive/poseclassifier/training'
path_tempfiles = '/content/drive/My Drive/CSE535Temp/'
path_classifydata = '..\\..\\flaskr\\uploads\\classify'
list_gesture = ['BUY', 'COMMUNICATE', 'FUN', 'HOPE', 'MOTHER', 'REALLY']
#list_excludedparts = ['leftKnee', 'rightKnee', 'leftAnkle', 'rightAnkle']
list_excludedparts = []
                      # +
listpartsinuse = ['nose', 'leftEye','rightEye','leftEar', 'rightEar',
                      'leftShoulder','rightShoulder', 'leftHip', 'rightHip',
                      'leftKnee', 'rightKnee', 'leftAnkle', 'rightAnkle',
                      'leftElbow', 'rightElbow', 'leftWrist', 'rightWrist'
                      ]             


#list_gesture = ['BUY', 'HOUSE', 'FUN', 'HOPE', 'ARRIVE', 'REALLY', 'READ', 'LIP', 'MOUTH', 'SOME', 'COMMUNICATE', 'WRITE', 'CREATE', 'PRETEND', 'SISTER', 'MAN', 'ONE', 'DRIVE', 'PERFECT', 'MOTHER']
#list_gestureindex = [i.key for i in enumerate(list_gesture)]
ftst = []
ftrn = []
ftrn2 = []
dict_tst = {}
dict_trn = {}
dict_trn2 = {}


# Feature Development

In [ ]:

def plainfunc(lst):
    # print('plain')
    return [lst, ['plot', '']]


def fftfunc(lst):
    fftresult = np.abs(fft(lst))
    half = len(fftresult) // 2
    return [fftresult[half:], ['stem', '']]


def meanfunc(lst):
    return [[sum(lst) / float(len(lst))], ['plot', 'o']]


def stdfunc(lst):
    return [[np.std(lst)], ['plot', 'o']]


def varfunc(lst):
    return [[np.var(lst)], ['plot', 'o']]


def dwtmultifunc(lst):
    # print('dwting')
    # w = pywt.Wavelet('db2')
    # lst2 = map(int,lst)
    #    cA, cD = pywt.dwt(lst, wavelet=w, mode='smooth')
    w = pywt.Wavelet('db1')
    cData = []
    lvl = 4
    if pywt.dwt_max_level(len(lst), w) < lvl:
        lvl = pywt.dwt_max_level(len(lst), w)
    cData = pywt.wavedec(lst, w, level=lvl)  # wavelet=w, mode='smooth')
    return [cData[0], ['plot', '']]
    # return dwt(lst)


def dwtbasicfunc(lst):
    # =============================================================================
    #     for family in pywt.families():
    #         print("%s family: " % family + ', '.join(pywt.wavelist(family)))
    # =============================================================================
    # w = pywt.Wavelet('db2')
    # lst2 = map(int,lst)
    #    cA, cD = pywt.dwt(lst, wavelet=w, mode='smooth')
    w = pywt.Wavelet('db2')
    cData = pywt.dwt(lst, w)  # wavelet=w, mode='smooth')
    return [cData[0], ['plot', '']]


def rmsfunc(lst):
    return [[np.sqrt(np.mean(np.square(lst)))], ['plot', 'o']]

def maxfunc(lst):
    return [[np.max(lst)], ['plot', 'o']]

def minfunc(lst):
    return [[np.min(lst)], ['plot', 'o']]

def energyfunc(lst):
    # =============================================================================
    #     # Rearrange x into 10 30 second windows
    #     x = sp.reshape(lst, (-1, 30))
    #     # Calculate power over each window [J/s]
    #     p = sp.sum(x*x, 1)/x.size
    #     # Calculate energy [J = J/s * 30 second]
    #     e = p*x.size
    # =============================================================================
    # print('energy')
    e = sum(np.square(np.absolute(lst)))
    return [[e], ['plot', 'o']]

def dNRD(lst):
    return zCRMaxDfunction(lst, 'DNRD')

def zCR(lst):
    return zCRMaxDfunction(lst, 'ZCR')

def mDA(lst):
    return zCRMaxDfunction(lst, 'MDA')

def ZMD(lst):
    return zCRMaxDfunction(lst, 'ALL')


def zCRMaxDfunction(lst, func):
    #Source: Professor's code
    diffNormRawData = np.diff(lst)

    zeroCrossingArray = np.array([])
    maxDiffArray = np.array([])

    if diffNormRawData[0] > 0:
        initSign = 1
    else:
        initSign = 0

    windowSize = 5;

    for x in range(1, len(diffNormRawData)):
        if diffNormRawData[x] > 0:
            newSign = 1
        else:
            newSign = 0

        if initSign != newSign:
            zeroCrossingArray = np.append(zeroCrossingArray, x)
            initSign = newSign
            maxIndex = np.minimum(len(diffNormRawData), x + windowSize)
            minIndex = np.maximum(0, x - windowSize)

            maxVal = np.amax(diffNormRawData[minIndex:maxIndex])
            minVal = np.amin(diffNormRawData[minIndex:maxIndex])

            maxDiffArray = np.append(maxDiffArray, (maxVal - minVal))

    index = np.argsort(-maxDiffArray)

    itm1 = list(diffNormRawData) if func in ['DNRD', 'ALL'] else []
    itm2 = list(zeroCrossingArray[index[0:5]]) if len(zeroCrossingArray) > 0 else 0.0
    itm2 = itm2 if func in ['ZCR', 'ALL'] else []
    itm3 = list(maxDiffArray[index[0:5]]) if len(maxDiffArray) > 0 else 0.0
    itm3 = itm3 if func in ['MDA', 'ALL'] else []
    #print(itm1, itm2, itm3)
    return [[itm1, itm2, itm3], ['plot', 'o']]

def clip(l, limit):
    """Clips an sequence to the central *limit* values."""
    lo = hi = (len(l) - limit) / 2
    lo = math.ceil(lo)
    hi = math.floor(hi)
    return l[lo:-hi]

def developfeaturematrix(features, dataset, aggregate, limit):
    #masterdataset = dataset
    #tempdataset = []
    ftroptions = {
        'plain': plainfunc,
        'energy': energyfunc,
        'fft': fftfunc,
        'mean': meanfunc,
        'std': stdfunc,
        'var': varfunc,
        'dwt': dwtbasicfunc,
        'dwtMulti': dwtmultifunc,
        'rms': rmsfunc,
        'max': maxfunc,
        'min': minfunc,
        'zcr': zCR,
        'mda': mDA,
        'dnrd': dNRD,
        'zmd': ZMD
        #'box': boxplotfunc,

    }

    listcolumns = [col for col in dataset.columns if col not in ['Gesture', 'Sample', 'Frame']]

    testarr = []
    listofrowstoadd = np.array([])
    rowslist = []
    # samplerangelimit = 4 if activity == "TRAIN" else 2
    if not aggregate:
        featmatrix = []
        onerow = []
        for i_gesture in range(len(list_gesture)):
            print("Feature Creation for Gesture : " + str(i_gesture))
            totsam = len(dataset[(dataset.Gesture == i_gesture)].Sample.unique())
            for x_sample in range(totsam):
                onerow = [i_gesture, x_sample]

                for col in listcolumns:
                    lst = dataset.loc[(dataset.Gesture == i_gesture) & (dataset.Sample == x_sample), col].values
                    #lst = dataset.loc[(dataset.Gesture == i_gesture & dataset.Sample == x_sample), col].to_numpy()
                    #print("Len orig: ", len(lst))
                    lst = clip(lst, limit)
                    #print("Len new: ", len(lst))
                    for key, _ in ftroptions.items():
                        if key in features:
                            #print("feature - " + key)
                            rslt = ftroptions[key](lst)
                            #testarr.append([i_gesture, x_sample, col, key, rslt[0]])
                            rslt2 = flatten2list(rslt[0])
                            onerow.extend(rslt2)
                            #print(key)
                featmatrix.append(flatten2list(onerow))
                #onerow = []
    else:
        print("aggregate style")
        featmatrix = []
        onerow = []
        for i_gesture in range(len(list_gesture)):
            print("Feature Creation for Gesture : " + str(i_gesture))
            totsam = len(dataset[(dataset.Gesture == i_gesture)].Sample.unique())
            for x_sample in range(totsam):
                setdata = dataset.loc[(dataset.Gesture == i_gesture) & (dataset.Sample == x_sample), listcolumns]
                onerow = [i_gesture, x_sample]
                #framestart = np.min(setdata['Frame'])
                frameend = np.max(setdata['Frame'])
                onerow.extend(setdata.mean(axis=1, skipna=True))
                #for frame in range(0, 1 + frameend):
                featmatrix.append(flatten2list(onerow))


    return np.array(featmatrix)

#def flattennestedlist(lst):
##    return [item for sublist in lst for item in sublist] if type(lst) == list else

#https://symbiosisacademy.org/tutorial-index/python-flatten-nested-lists-tuples-sets/
def flatten2list(object):
    gather = []
    for item in object:
        if isinstance(item, (list, tuple, set)):
            gather.extend(flatten2list(item))
        else:
            gather.append(item)
    return gather
### +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

def runpca(matrix, perc):
    #pcamatrix = np.array(matrix)
    classcol = np.array(list(zip(*matrix))[0])
    # # Y_train = [list_gesture.index(i) for i in Y_train]
    pcamatrix = np.array([item[1 - len(item):] for item in matrix])
    #numcol = matrix.shape[1]
    #usercol = matrix[:, numcol - 1]
    #pcamatrix = np.delete(matrix, numcol - 1, 1)
    #numcol = pcamatrix.shape[1]
    #classcol = pcamatrix[:, numcol - 1]


    #pcamatrix = np.delete(pcamatrix, numcol - 1, 1)
    # print('Original Matrix \n%s' %pd.DataFrame(matrix))
    # componentnum = 6
    # print(matrix)
    # mrowlen = len(matrix[0])
    # X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
    X_std = skprep.StandardScaler().fit_transform(pcamatrix)

    # print('Original Matrix \n%s' %pd.DataFrame(X_std))
    # pca = PCA(n_components=componentnum) #just using it for comparison
    # rslt = pca.fit(X_std)
    # rslt2 = pca.fit_transform(X_std) #this comes back with the proper rows to match
    # Yeigvec = rslt.components_
    # Yeigval = rslt.explained_variance_

    # =============================================================================
    # print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))
    cov_mat = np.cov(X_std.T)

    eig_vals, eig_vecs = np.linalg.eig(cov_mat)

    # print('Eigenvectors \n%s' %pd.DataFrame(eig_vecs))
    # print('\nEigenvalues \n%s' %pd.DataFrame(eig_vals))

    # Make a list of (eigenvalue, eigenvector) tuples
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i]) for i in range(len(eig_vals))]

    # Sort the (eigenvalue, eigenvector) tuples from high to low
    # eig_pairs.sort()
    # eig_pairs.reverse()

    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    # Visually confirm that the list is correctly sorted by decreasing eigenvalues
    # print('Eigenvalues in descending order:')
    # for i in eig_pairs:
    #    print(i[0])

    exp_var_percentage = perc  # Threshold of explained variance
    tot = sum(eig_vals)
    var_exp = [(i / tot) * 100 for i in sorted(eig_vals, reverse=True)]
    cum_var_exp = np.cumsum(var_exp)

    num_vec_to_keep = 0

    for index, percentage in enumerate(cum_var_exp):
        if percentage > exp_var_percentage:
            num_vec_to_keep = index + 0
            break
    print("number of vec to keep=%s" % num_vec_to_keep)

    # Compute the projection matrix based on the top eigen vectors
    num_features = X_std.shape[1]
    proj_mat = eig_pairs[0][1].reshape(num_features, 1)
    for eig_vec_idx in range(1, num_vec_to_keep):
        proj_mat = np.hstack((proj_mat, eig_pairs[eig_vec_idx][1].reshape(num_features, 1)))

    # Project the data
    pca_data = X_std.dot(proj_mat)
    # newnumcol = pca_data.shape[1]
    myclasscol = [[i] for i in classcol]
    pca_data = np.append(pca_data, myclasscol, 1)

    #myusercol = [[i] for i in usercol]
    #pca_data = np.append(pca_data, myusercol, 1)

    # print('New VERSION Feature Matrix: \n%s' %pd.DataFrame(pca_data))

    print("PCA Done - New Feature Matrix created")
    return pca_data


# Data preprocessing

In [ ]:

def printcharts(dataset, activity):
    fontsize = 10
    pltsize = 18
    x = 0
    #plt.rcParams.update({'font.size': fontsize})
    listcolumns = [col for col in dataset.columns if col not in ['Gesture', 'Sample', 'Frame']]
    #listcolumns = listcolumns[:6]
    list_gest = list_gesture#[:1]
    fig, axes = plt.subplots(24, 2, figsize=(pltsize, 24*4*4*2))
    rowcnt = 0
    for i_gesture in range(len(list_gest)):
            for i, col in enumerate(sorted(listcolumns)):
                rowcnt = rowcnt + 1 if x == 1 else rowcnt
                x = i % 2

                for x_sample in range(len(dataset[(dataset.Gesture == i_gesture)].Sample.unique())):
                    yvalues = dataset.loc[(dataset.Gesture == i_gesture) & (dataset.Sample == x_sample), col].values
                    xvalues = [i for i in range(len(yvalues))]
                    axes[rowcnt, x].set_title("Gesture"+ str(i_gesture) + " - " + col, loc='left', fontsize=8, position=(0.0,0.1))
                    #axes[rowcnt, x].set_ylabel('Values')
                    axes[rowcnt, x].grid(True)
                    axes[rowcnt, x].yaxis.set_ticks([])
                    plt.yticks(xvalues, "")
                    lines, = axes[rowcnt, x].plot(xvalues, yvalues)
                    plt.setp(lines, linewidth=1.0)
                #rowcnt+=1


                    #setdata = datatbl.loc[(datatbl.Gesture == i_gesture) & (datatbl.Sample == x_sample)]

    #fig.subplots_adjust(top=.95, hspace=.5, wspace=0.2)
    #fig.tight_layout()
    plt.show()

    fig.savefig("Chart"+activity+".png")
    print(listcolumns)

def createtable(data):
    dataset = []
    i = 0
    j = 0
    for gesturekey, gesturevalues in data.items():
        if 'PRACTICE' in gesturekey:
            samplename = int(gesturekey.split('_')[2])
            gesturename = gesturekey.split('_')[0] 
        else:
            samplename = int(gesturekey.split('-')[1])
            gesturename = gesturekey.split('-')[0]
        for frame in gesturevalues:
            keypoints = frame['keypoints']
            j = 0
            for point in keypoints:
                if point['part'] not in list_excludedparts:
                    dataset.append([list_gesture.index(gesturename), samplename, i, str('{:02d}'.format(j) +'-'+ point['part']), point['position']['x'], point['position']['y']])
                    j += 1
            i += 1
        i = 0
    #print(dataset.loc[(dataset.Gesture == 0)])
    return pd.DataFrame(dataset, columns=['Gesture', 'Sample', 'Frame', 'Part', 'X', 'Y'])

def fixcolumn(choice, txt):
    if choice == "partswide":
        return txt if '-' not in txt else txt.replace("(","").replace(")","").replace("'","").split(', ')[1] + "_" + txt.replace("(","").replace(")","").replace("'","").split(', ')[0]
    else:
        return txt if '-' not in txt else txt.replace("(","").replace(")","").replace("'","").split(', ')[0] + "_Frm_" + str('{:03d}').format(int(txt.replace("(","").replace(")","").replace("'","").split(', ')[1]))

def sortcolumns(numbasic, lst):
    basiclst = []
    worklst = []
    
    for i in range(len(lst)):
        if i <= (int(numbasic) - 1):
            basiclst.append(lst[i])
        else:
            worklst.append(lst[i])
    basiclst.extend(sorted(worklst))
    #print(basiclst)
    return basiclst

def normalize_data(partswide_datatbl, activity):
    global listpartsinuse
    listcolumns = [col for col in partswide_datatbl.columns if col not in ['Part', 'Gesture', 'Sample', 'Frame']]
    #listpartsinuse = ['02-leftWrist', '03-rightWrist'] #'00-leftElbow', '01-rightElbow',
    print("Normalizing Data")
    print(partswide_datatbl.head)
    #samplerangelimit = 4 if activity == "TRAIN" else 2
    for i_gesture in range(len(list_gesture)):
        for x_sample in range(len(partswide_datatbl[(partswide_datatbl.Gesture == i_gesture)].Sample.unique())):
            for col in partswide_datatbl.columns:
                if col in listcolumns:
                    for part1 in listpartsinuse:
                        try:
                            workdata = partswide_datatbl.loc[(partswide_datatbl.Gesture == i_gesture) & (partswide_datatbl.Sample == x_sample) & (partswide_datatbl.Part == part1), col].values#.filter(items=[col])
                            workdata2 = (workdata - np.mean(workdata))/((np.max(workdata-np.mean(workdata))-np.min(workdata-np.mean(workdata)))+0.0000000001)
                            
                            partswide_datatbl.loc[(partswide_datatbl.Gesture == i_gesture) & (partswide_datatbl.Sample == x_sample) & (partswide_datatbl.Part == part1), col] = workdata2
                        except ValueError:  #raised if `y` is empty.
                            #print("Value data: " % (i_gesture, x_sample))#, part1 ))
                            print(workdata)
                            pass
                        
    return partswide_datatbl

def distancecalc(X1, Y1, X2, Y2):
    return  np.linalg.norm(np.array(X1, Y1)- np.array(X2, Y2))

def originshiftcalc(dyn, stat1, stat2):
    return (stat1.values[0] - dyn)/(stat1.values[0] - stat2.values[0])

def normalize_to_nose(datatbl2, activity):
    #listcolumns = [col for col in partswide_datatbl.columns if col not in ['Gesture', 'Sample', 'Frame', 'OverallScore'] and 'PartScore' not in col ]
    #varcols = ['X','Y']
    global listpartsinuse
    #print(datatbl2.shape)
    #print(datatbl2.loc[(datatbl2.Gesture == 0)])
    datatbl = np.array(datatbl2)
    #samplerangelimit = 4 if activity == "TRAIN" else 2
    for i_gesture in range(len(list_gesture)):
        for x_sample in range(len(np.unique(datatbl[datatbl[:,0] == i_gesture][:,1]))):
        #for x_sample in range(len(datatbl[(datatbl.Gesture == i_gesture)].Sample.unique())):
            # for col in datatbl.columns:
            #     if col in varcols:
            print(i_gesture, x_sample)
            #print(datatbl[datatbl[:,1] == x_sample])
            #print(datatbl[(datatbl[:,0] == i_gesture) & (datatbl[:,1] == x_sample)])
            setdata = datatbl[(datatbl[:,0] == i_gesture) & (datatbl[:,1] == x_sample)]
            framestart = np.min(setdata[:,2])
            frameend = np.max(setdata[:,2])
            #setdata = datatbl.loc[(datatbl.Gesture == i_gesture) & (datatbl.Sample == x_sample)]
            # framestart = np.min(setdata['Frame'])
            # frameend = np.max(setdata['Frame'])
            for i in range(framestart, frameend + 1):
                workdata = setdata[setdata[:,2]== i]
                nosedata = workdata[workdata[:,3] == '00-nose']
                hipdata = workdata[workdata[:,3] == '11-leftHip']
                
                #workdata = setdata.loc[(setdata.Frame == i)] #(setdata.Gesture == i_gesture) & (setdata.Sample == x_sample &
                #nosedata = workdata.loc[(workdata.Part == '00-nose')]
                #hipdata = workdata.loc[(workdata.Part == '11-leftHip')]
                #for row in workdata:
                #print(i_gesture, x_sample, i)
                #print(workdata)
                #print(nosedata)
                #print(hipdata)

              
                updatedcol = (nosedata[0][4] - workdata[:,4])/(nosedata[0][4] - hipdata[0][4])
                #print(updatedcol.shape)
                #print(updatedcol)
                #print("xxxxxxxxxxxxxx")
                #print(datatbl2.loc[(datatbl2.Gesture == i_gesture) & (datatbl2.Sample == x_sample) & (datatbl2.Frame == i), 'X'])
                #datatbl[(datatbl[:,0] == i_gesture) & (datatbl[:,1] == x_sample) & (datatbl[:,2] == i)][:,4] = updatedcol
                datatbl2.loc[(datatbl2.Gesture == i_gesture) & (datatbl2.Sample == x_sample) & (datatbl2.Frame == i), 'X'] = updatedcol
                #np.put(datatbl[(datatbl[:,0] == i_gesture) & (datatbl[:,1] == x_sample) & (datatbl[:,2] == i)],[:,4], updatedcol )
                #print("yyyyyyyyyyyyyy")
                #print(datatbl2.loc[(datatbl2.Gesture == i_gesture) & (datatbl2.Sample == x_sample) & (datatbl2.Frame == i), 'X'])

                updatedcol = (nosedata[0][5] - workdata[:,5])/(nosedata[0][5] - hipdata[0][5])
                #print(updatedcol.shape)
                #print(updatedcol)
                #print("xxxxxxxxxxxxxx")
                #print(datatbl2.loc[(datatbl2.Gesture == i_gesture) & (datatbl2.Sample == x_sample) & (datatbl2.Frame == i), 'Y'])
                datatbl2.loc[(datatbl2.Gesture == i_gesture) & (datatbl2.Sample == x_sample) & (datatbl2.Frame == i), 'Y'] = updatedcol
                #np.put(datatbl[(datatbl[:,0] == i_gesture) & (datatbl[:,1] == x_sample) & (datatbl[:,2] == i)], [:,5], updatedcol)
                #print("yyyyyyyyyyyyyy")
                #print(datatbl2.loc[(datatbl2.Gesture == i_gesture) & (datatbl2.Sample == x_sample) & (datatbl2.Frame == i), 'Y'])
                
                #print((nosedata[0] - workdata)/(nosedata[0] - hipdata[0]))

                #-------------------
                #npworkdata = setdata.loc[(setdata.Frame == i)].values
                
                #for nppart in npworkdata[npworkdata[4]]
                #print(datatbl.loc[(datatbl.Gesture == i_gesture) & (datatbl.Sample == x_sample) & (setdata.Frame == i)])
                #datatbl.loc[(datatbl.Gesture == i_gesture) & (datatbl.Sample == x_sample) & (setdata.Frame == i), 'X'] = workdata['X'].apply(originshiftcalc, args=(nosedata['X'], hipdata['X']))
                #datatbl.loc[(datatbl.Gesture == i_gesture) & (datatbl.Sample == x_sample) & (setdata.Frame == i), 'Y'] = workdata['Y'].apply(originshiftcalc, args=(nosedata['Y'], hipdata['Y']))
                #shoulderleftdata = workdata.loc[(workdata.Part == '05-leftShoulder')]
                #shoulderrightdata = workdata.loc[(workdata.Part == '06-leftShoulder')]

                #datatbl.loc[(datatbl.Gesture == i_gesture) & (datatbl.Sample == x_sample) & (setdata.Frame == i), 'X'] = workdata['X']
                #datatbl.loc[(datatbl.Gesture == i_gesture) & (datatbl.Sample == x_sample) & (setdata.Frame == i), 'Y'] = workdata['Y']
            # for col in partswide_datatbl.columns:
            #     if col in listcolumns:
            #         workdata = partswide_datatbl.loc[(partswide_datatbl.Gesture == i_gesture) & (partswide_datatbl.Sample == x_sample), col]#.filter(items=[col])
            #         workdata = (workdata - np.mean(workdata))/(np.max(workdata-np.mean(workdata))-np.min(workdata-np.mean(workdata)))
            #         partswide_datatbl.loc[(partswide_datatbl.Gesture == i_gesture) & (partswide_datatbl.Sample == x_sample), col] = workdata
    return datatbl

def createtable_partswide(basicdata): 
    pivoteddataset = basicdata.pivot_table(['X','Y'], ['Gesture', 'Sample', 'Frame'], 'Part',fill_value=0)
    flattened = pd.DataFrame(pivoteddataset.to_records())
    flattened.columns = [fixcolumn('partswide', col) for col in flattened.columns]
    flattened = flattened.reindex(sortcolumns(4, flattened.columns), axis=1)
    
    # normalized = normalize_data(flattened, "TRAIN")
    # return normalized
    return flattened

def createlist_frameswide(maxframecnt, partswide):
    listcolumns = [col for col in partswide.columns if col not in ['Gesture', 'Sample', 'Frame']]


    testarr = []
    listofrowstoadd = np.array([])
    rowslist = []
    #samplerangelimit = 4 if activity == "TRAIN" else 2
    for i_gesture in range(len(list_gesture)):
        totsam = len(partswide[(partswide.Gesture == i_gesture)].Sample.unique())
        for x_sample in range(totsam):
            currlen = len(partswide[(partswide.Gesture == i_gesture) & (partswide.Sample == x_sample)])
            padlength = maxframecnt - currlen

            #testarr.append([[v[0], v[1], currlen + k].extend(np.zeros(24)) for k, v in enumerate(padlength * [[i_gesture, x_sample]])])
            for i_pad in range(padlength):


            #listofrowstoadd.extend(testarr)
                testarr.append(np.array([i_gesture, x_sample, currlen + i_pad,  # np.zeros(34)
                                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
                                                     ]))
                # np.append(listofrowstoadd, np.array([i_gesture, x_sample, currlen + i_pad, #np.zeros(34)
                #                                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                #                                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                #                                  ]))
                # listofrowstoadd.append(pd.Series([i_gesture, x_sample, currlen + i_pad,  # np.zeros(34)
                #                                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                #                                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
                #                                   ]), ignore_index=True)
    print("Just before sort/add - time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
    if len(testarr) > 0:
        partswide = partswide.append(pd.DataFrame(np.array(testarr), columns=partswide.columns), ignore_index=True)
        partswide.sort_values(by=['Gesture', 'Sample', 'Frame'], inplace=True)
        partswide = partswide.reset_index(drop=True)
    print("Midway through frameswide. Pivoting time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
    #pivoteddataset = partswide.pivot_table(listcolumns, ['Gesture', 'Sample', 'OverallScore'], 'Frame')
    #flattened = pd.DataFrame(pivoteddataset.to_records())
    #flattened.columns = [fixcolumn('frameswide', col) for col in flattened.columns]
    #flattened = flattened.reindex(sortcolumns(3, flattened.columns), axis=1)
    #####################
    # totalset = []
    # onerec = []
    # for i_gesture in range(len(list_gesture)):
    #     totsam = len(partswide[(partswide.Gesture == i_gesture)].Sample.unique())
    #     for x_sample in range(1, 1+ totsam):
    #         onerec = [i_gesture, x_sample]
    #         for colnam in listcolumns:
    #             onerec.extend(partswide.loc[(partswide.Gesture == i_gesture) & (partswide.Sample == x_sample), colnam].tolist())
    #         totalset.append(onerec)
    #         onerec = []

    pivoteddataset = partswide.pivot_table(listcolumns, ['Gesture', 'Sample'], 'Frame', fill_value=0)
    flattened = pd.DataFrame(pivoteddataset.to_records())
    flattened.columns = [fixcolumn('frameswide', col) for col in flattened.columns]
    flattened = flattened.reindex(sortcolumns(3, flattened.columns), axis=1)

    # normalized = normalize_data(flattened, "TRAIN")
    # return normalized
    return flattened
    #newframecnt = checkmaxframecount([partswide])
    #print("new max frame count is: "+newframecnt)
    #return flattened
    ###return totalset
                
def padlengths(featuredatasets):
    maxlencnt = 0
    new_featuredatasets = []#np.array()
    
    # for arr in featuredatasets:
    #     for line in arr:
    #         # for i_gesture in range(len(list_gesture)):
    #         #     for x_sample in range(len(partswide[(partswide.Gesture == i_gesture)].Sample.unique())):
    #         #         setlength = len(partswide[(partswide.Gesture == i_gesture) & (partswide.Sample == x_sample)])
    #         setlength = len(line)
    #         if maxlencnt < setlength:
    #             maxlencnt = setlength
    for arr in featuredatasets:
        if len(arr) > 0:
          length = max(map(len, arr))
          if maxlencnt < length:
              maxlencnt = length

    for arr2 in featuredatasets:
        new_featuredatasets.append(np.array([xi + [0] * (maxlencnt - len(xi)) for xi in arr2]))
        # for line in arr:
        #     currlen = len(line)
        #     padlength = maxlencnt - currlen
        #     #line.extend(np.zeros(padlength))
        #     line.pad(np.zeros(padlength))

    print("Max Len count" + str(maxlencnt))
    return new_featuredatasets

### +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
### +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
### +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

def checkdatafile(filename):
    return path.exists(path_tempfiles+filename + '_export.csv')

def savedatafile(pd, filename):
    pd.to_csv(r''+path_tempfiles+filename+'_export.csv', index = True, header=True)

def readdatafile(filename):
    return pd.read_csv(r''+path_tempfiles+filename+'_export.csv', index_col = 0, header=0)

def savenumpydata(npdata, filename):
    np.savetxt(r''+path_tempfiles+filename+'_export.csv', npdata, delimiter = ',')

def readnumpydata(filename):
    return np.loadtxt(r''+path_tempfiles+filename+'_export.csv', delimiter=',')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def createClassifierData(test_data, train_data):


    Y_train = np.array(list(zip(*train_data))[0])
    # # Y_train = [list_gesture.index(i) for i in Y_train]
    X_train = [item[1 - len(item):] for item in train_data]
    #
    Y_test = np.array(list(zip(*test_data))[0])  # test_data[:,0]
    X_test = [item[1 - len(item):] for item in test_data]  # test_data[:,2:]

    #X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.20, random_state = 42)

    return X_test, Y_test, X_train, Y_train

def preparedata(features, forcenewfile, forcenewfeatures, limit):
    print("Data Preparation starting time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
    versionnum = "_vpost1"
    
    if not (checkdatafile("test_original"+versionnum) and forcenewfile == False):
        for (dirpath, dirnames, filenames) in walk(path_testdata):
            ftst.append([dirpath, dirnames, filenames])
        
        # for (dirpath, dirnames, filenames) in walk(path_trainingdata):
        #     ftrn.append([dirpath, dirnames, filenames])
            
        for (dirpath, dirnames, filenames) in walk(path_trainingdata2):
            ftrn2.append([dirpath, dirnames, filenames])
            
        #print("len "+ str(len(ftst[0][1])))
        #print(ftst)
        #print(ftrn2)
        for i in range(len(ftst[0][1])):
            if str(ftst[0][1][i]).split('-')[0] in list_gesture:
                #print(ftst[i + 1][0]+'/key_points.json')
                dict_tst[str(ftst[0][1][i]).split('-')[0] + "-0"] = json.load(open(ftst[i + 1][0]+'/key_points.json'))
                #dict_tst[str(ftst[0][1][i]).split('-')[0]] = json.load(open(ftst[i + 1][0]+'\key_points.json'))
            
        # for i in range(len(ftrn[0][1])):
        #     if str(ftrn[0][1][i].split('_')[0]) in list_gesture:
        #         dict_trn[str(ftrn[0][1][i]).upper()] = json.load(open(ftrn[i + 1][0]+'\key_points.json'))   
        #print("HELLO")            
        for i in range(len(ftrn2[0][1])):
            if str(ftrn2[0][1][i].upper().split('_')[0]) in list_gesture: 
                for x in range(1, len(ftrn2)):
                    #print(str(ftrn2[0][1][i].upper().split('_')[0]))
                    if str(ftrn2[0][1][i].upper().split('_')[0]) in ftrn2[x][0].upper():
                        for cnt in range(len(ftrn2[x][2])):
                            dict_trn2[str(ftrn2[0][1][i].upper().split('_')[0])+"-" + str(cnt)] = json.load(open(ftrn2[x][0]+ "/" + ftrn2[x][2][cnt]))
                #print(str(ftrn2[0][1][i].split('_')[0]))

    
    #print("Basic Data start time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
    ##------------------------------------------
    #if checkdatafile("test_original"+versionnum) and forcenewfile == False:
    #    pd_tst1 = readdatafile("test_original"+versionnum)
    #else:
    #    pd_tst1 = createtable(dict_tst)
    #    savedatafile(pd_tst1, "test_original" + versionnum) 
   
   
    print("Data Normalization start time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
    #------------------------------------------
    if checkdatafile("test_normalized"+versionnum) and forcenewfile == False:
        pd_tst = readdatafile("test_normalized"+versionnum)
    else:
        pd_tst1 = createtable(dict_tst)
        #savedatafile(pd_tst1, "test_original" + versionnum)
        #pd_tst = pd_tst1
        #pd_tst = normalize_to_nose(pd_tst1,'TEST')
        pd_tst = normalize_data(pd_tst1, "TEST")
        savedatafile(pd_tst, "test_normalized"+versionnum)
    #------------------------------------------
    if checkdatafile("train_normalized"+versionnum) and forcenewfile == False:
        pd_trn = readdatafile("train_normalized"+versionnum)
    else:
        pd_trn2 = createtable(dict_trn2)
        #savedatafile(pd_trn2, "train_original" + versionnum)
        #pd_trn = pd_trn2
        #pd_trn = normalize_to_nose(pd_trn2, 'TRAIN')
        pd_trn = normalize_data(pd_trn2, "TRAIN")
        savedatafile(pd_trn, "train_normalized"+versionnum)
    #------------------------------------------
    #pd_trn2 = createtable(dict_trn2)
    #pd_trn2 = normalize_data(pd_trn2, "TRAIN")
    #pd_trn = normalize_to_nose(pd_trn, 'TRAIN')
    #------------------------------------------
    #print(pd_trn.head(100))
    print("Data Re-Shaping start time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))


    if checkdatafile("test_partswide"+versionnum) and forcenewfile == False:
        pd_tst_parts = readdatafile("test_partswide"+versionnum)
    else:
        pd_tst_parts = createtable_partswide(pd_tst)
        savedatafile(pd_tst_parts, "test_partswide"+versionnum)

    if checkdatafile("train_partswide"+versionnum) and forcenewfile == False:
        pd_trn_parts = readdatafile("train_partswide"+versionnum)
    else:
        pd_trn_parts = createtable_partswide(pd_trn)
        savedatafile(pd_trn_parts, "train_partswide"+versionnum)

    print("Printing charts at time - {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
    #printcharts(pd_tst_parts, "test")
    #printcharts(pd_trn_parts, "train")


    # if checkdatafile("test_features"+versionnum) and 
    if checkdatafile("train_features"+versionnum) and forcenewfeatures == False:
        np_tst_parts_features = readnumpydata("test_features"+versionnum)
        np_trn_parts_features = readnumpydata("train_features"+versionnum)
        print("Pulled Re-shaped Data ending time at {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
    else:
        print("Data 1 Re-shaping Test Frames Wide starting time is {}".format(
            datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
        np_tst_parts_features = developfeaturematrix(features, pd_tst_parts, False, limit)
        print("Data 2 Re-shaping Train Frames Wide starting time is {}".format(
            datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
        np_trn_parts_features = developfeaturematrix(features, pd_trn_parts, False, limit)

        print("Padding Lengths starting time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
        np_tst_parts_features, np_trn_parts_features = padlengths([np_tst_parts_features, np_trn_parts_features])
        savenumpydata(np_tst_parts_features, "test_features"+versionnum)
        savenumpydata(np_trn_parts_features, "train_features"+versionnum)

    #print("RUN PCA starting time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
    #np_tst_parts_features = runpca(np_tst_parts_features, 60)
    #np_trn_parts_features = runpca(np_trn_parts_features, 60)
    

    return np_tst_parts_features, np_trn_parts_features




# Classifiers

In [ ]:
def classify_ann(X_test, Y_test, X_train, Y_train):
    #print("Train shape: " + np.array(train_data).shape)
    #print("Test shape: " + np.array(test_data).shape)
    #Y_train = train_data[:,0]
    #https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
    num_hidden = int(len(X_test[0])/2)

    parameter_space = {
    'hidden_layer_sizes': [(num_hidden, round(num_hidden/2), round(num_hidden/4)), (num_hidden,num_hidden*2,num_hidden), (num_hidden,)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam'],
    #'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': [0.0001, 0.0005,0.001]
    }

    #from sklearn.preprocessing import StandardScaler
    #scaler = StandardScaler()
    #scaler.fit(X_train)
    
    #X_train = scaler.transform(X_train)
    #X_test = scaler.transform(X_test)
    from sklearn.neural_network import MLPClassifier  
    mlp = MLPClassifier(max_iter=800, verbose=True, tol=0.000000100, momentum=0.9,early_stopping=False)
    #37-44% range
    # mlp = MLPClassifier(hidden_layer_sizes=(num_hidden, round(num_hidden/2), round(num_hidden/4)), max_iter=800, activation='logistic', momentum=0.9,early_stopping=False,
    #                     learning_rate='adaptive', verbose=True, tol=0.000000100, learning_rate_init=0.0001)

    clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=2, verbose=1, )
    clf.fit(X_train, Y_train)

    # Best parameters set
    print('Best parameters found:\n', clf.best_params_)

    # All results
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

    predictions = clf.predict(X_test)


    pickle.dump(mlp, open("model_ANN.pkl", 'wb'))
    print(classification_report(Y_test,predictions, target_names=list_gesture))


    acc = accuracy_score(Y_test, predictions)
    print("Accuracy: "+str(acc))
    rslt2 = precision_recall_fscore_support(Y_test,predictions)
    
def classify_NB(X_test, Y_test, X_train, Y_train):


    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB().fit(X_train, Y_train)
    gnb_predictions = gnb.predict(X_test)
    pickle.dump(gnb_predictions, open("model_NB.pkl", 'wb'))
    accuracy = gnb.score(X_test, Y_test)

    print(classification_report(Y_test, gnb_predictions, target_names=list_gesture))
    acc = accuracy_score(Y_test, gnb_predictions)
    print("Accuracy: "+str(acc))
    rslt2 = precision_recall_fscore_support(Y_test,gnb_predictions)


def classify_KNN(X_test, Y_test, X_train, Y_train):
    from sklearn import neighbors
    clf = neighbors.KNeighborsClassifier(n_neighbors=2)
    clf.fit(X_train, Y_train)
    knn_predictions = clf.predict(X_test)
    pickle.dump(knn_predictions, open("model_KNN.pkl", 'wb'))

    print(classification_report(Y_test, knn_predictions, target_names=list_gesture))
    acc = accuracy_score(Y_test, knn_predictions)
    print("Accuracy: "+str(acc))
    rslt2 = precision_recall_fscore_support(Y_test,knn_predictions)

def classify_LSTMCNN(X_test, Y_test, X_train, Y_train):
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    # fix random seed for reproducibility
    np.random.seed(7)
    
    max_seq_length = 900
    X_train = sequence.pad_sequences(X_train, maxlen=max_seq_length)
    X_test = sequence.pad_sequences(X_test, maxlen=max_seq_length)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    print(X_train.shape)
    print(X_test.shape)
    print(Y_train.shape)
    print(Y_test.shape)

    #print(Y_train)
    #print(Y_test)
    orig_Y_train = Y_train
    orig_Y_test = Y_test

    Y_train = keras.utils.to_categorical(Y_train, num_classes=6)
    Y_test = keras.utils.to_categorical(Y_test, num_classes=6)
    model = Sequential()
    
    #model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    #model.add(MaxPooling1D(pool_size=2))
    
    model.add(Flatten())

    #model.add(LSTM(128, #return_sequences=True,
    #            input_shape=(max_seq_length, 1)))  # returns a sequence of vectors of dimension 32
    # #model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
    # #model.add(LSTM(32))  # return a single vector of dimension 32
    # #model.add(Dropout(0.2))
    model.add(LSTM(900))
    model.add(Dense(900, activation='relu'))
    model.add(Dense(450, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    # ####model.add(Flatten())
    # model.add(Dense(100, activation='relu'))
    # model.add(Dropout(0.2))
    # model.add(Dense(32, activation='relu'))
    # model.add(Dropout(0.2))
    # model.add(Dense(6, activation='softmax'))
    #model.add(Activation("softmax"))

    optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.build(X_train.shape)
    print(model.summary())
    print(model.optimizer.get_config())

    
    history = model.fit(X_train, Y_train, epochs=300, batch_size=415, verbose=0)
    # Final evaluation of the model

    yhat_probs = model.predict(X_test, verbose=0)
    # predict crisp classes for test set
    yhat_classes = model.predict_classes(X_test, verbose=0)

    print(orig_Y_test)
    print(yhat_classes)
    accuracy = accuracy_score(orig_Y_test, yhat_classes )
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(orig_Y_test, yhat_classes, average='micro' )
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(orig_Y_test, yhat_classes, average='micro' )
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(orig_Y_test, yhat_classes, average='micro' )
    print('F1 score: %f' % f1)
    #print(Y_test)
    #print(keras_predictions)
    #print(classification_report(Y_test, keras_predictions, target_names=list_gesture))

    

# Main

In [ ]:
##############################################################
#def main():

#features = ['plain','zCRMD','fft','dwt','mean','std','var','dwtMulti','rms','max','min','energy'] #36%
#features = [ 'plain', 'fft', 'dwt'] 37%
#features = [ 'fft', 'dwt', ] 45%
#Master List
#features = ['plain', 'fft','mean','std','var','dwt','dwtMulti','rms','max','min','energy', 'zcr', 'mda', 
#'dnrd', 'zmd']
#features = [ 'fft', 'dwt']
#features = [ 'zcr','mda', 'energy'] -61 on knn
#features = ['fft','dwt']
features = ['plain']

dataarrtst, dataarrtrn = preparedata(features, forcenewfile = True, forcenewfeatures = True, limit = 30)



X_test, Y_test, X_train, Y_train = createClassifierData(dataarrtst, dataarrtrn)

#print("Classification - ANN starting time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
#classify_ann(X_test, Y_test, X_train, Y_train)
# print("Classification - NB starting time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
# classify_NB(X_test, Y_test, X_train, Y_train)
# print("Classification - KNN starting time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
# classify_KNN(X_test, Y_test, X_train, Y_train)
print("Classification - LSTMCNN starting time is {}".format(datetime.datetime.now().strftime("%d-%b-%Y %H:%M:%S.%f")))
classify_LSTMCNN(X_test, Y_test, X_train, Y_train)
print("Finished")

# if __name__== "__main__":
#    main()

Data Preparation starting time is 24-Aug-2020 02:03:45.056338
Data Normalization start time is 24-Aug-2020 02:03:48.639192
Normalizing Data
<bound method NDFrame.head of       Gesture  Sample  Frame           Part           X           Y
0           2       0      0        00-nose  297.252092  122.229147
1           2       0      0     01-leftEye  303.581851  105.652602
2           2       0      0    02-rightEye  283.667632  109.216702
3           2       0      0     03-leftEar  323.020565  117.911668
4           2       0      0    04-rightEar  265.239934  121.279894
...       ...     ...    ...            ...         ...         ...
5163        4       0     42    12-rightHip  267.296056  356.657986
5164        4       0     42    13-leftKnee  341.240929  411.558842
5165        4       0     42   14-rightKnee  264.665384  417.398418
5166        4       0     42   15-leftAnkle  399.527739  363.993397
5167        4       0     42  16-rightAnkle  390.467750  362.994987

[5168 rows x 

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
Normalizing Data
<bound method NDFrame.head of         Gesture  Sample  Frame           Part           X            Y
0             2       0      0        00-nose  576.940031   397.228245
1             2       0      0     01-leftEye  605.434954   361.884875
2             2       0      0    02-rightEye  550.106654   362.770621
3             2       0      0     03-leftEar  663.817152   386.165365
4             2       0      0    04-rightEar  499.119996   397.913448
...         ...     ...    ...            ...         ...          ...
830938        0      70     63    12-rightHip  282.711698  1489.578359
830939        0      70     63    13-leftKnee  611.888606  1880.651919
830940        0      70     63   14-rightKnee  291.530753  1851.6874

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Streaming output truncated to the last 5000 lines.
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

ValueError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Bottom